Numba is a high performance compiler that translates functions to optimized machine code. This allows us to run numerical algorithms on a GPU without learning C or FORTRAN. Designed  to be used with NumPy for scientific computing.

The CUDA library from Numba supports GPU programming for a restricted subset of Python code in CUDA kernels allowing for parallelization of numerical algorithms. CUDA cores are available on Nvidia GPUs. Documentation can be found [here](https://numba.readthedocs.io/en/stable/cuda/overview.html).

Numba's GPU random number generator is an implementation of the xoroshiro128+ algorithm. Other random number generators are not supported by cuda.

The NumPy package provides a multidimensional array object along with functions providing functionality similar to MATLAB.

Matplot is a plotting library used with NumPy.

In [35]:
from numba import cuda
from numba.cuda.random import create_xoroshiro128p_states, xoroshiro128p_normal_float32
import numpy as np
import time
import matplotlib.pyplot as plt

The detect function from the cuda library will print a summary of supported CUDA hardware. This hardware must be present to use the CUDA framework.

In [36]:
cuda.detect()

Found 1 CUDA devices
id 0    b'NVIDIA GeForce RTX 4050 Laptop GPU'                              [SUPPORTED]
                      Compute Capability: 8.9
                           PCI Device ID: 0
                              PCI Bus ID: 1
                                    UUID: GPU-4093b37a-da12-a30d-cc27-f706fbbec7ec
                                Watchdog: Enabled
             FP32/FP64 Performance Ratio: 64
Summary:
	1/1 devices are supported


True

The below function computes the splitting probability of a diffusing particle on the interval [0, 1] leaving the interval at x = 0 conditioned on starting at 1/2. It is implemented in using NumPy functions and will run on the CPU.

In [37]:
def splitting_probability_cpu():
    x = 1/2
    while x > 0 and x < 1:
        x += 0.01 * np.random.normal()
    if x < 0:
        return 1
    else:
        return 0

To compare computation speed of CPU and GPU implementation we need to use CUDA's two-level hierarchy when declaring the number of threads. To do so we declare the number of blocks and the number of threads per block. The product of the two will give the total number of threads.

Each device has an optimal threads per block. Benchmark testing revealed mine to be 512. This is typically a multiple of 32.

In [38]:
threads_per_block = int(512)
blocks = int(10 ** 5 / threads_per_block)
trials = threads_per_block * blocks

The following CUDA kernel runs the same simulation on the GPU. This looks similar to the CPU implementation with a few important differences. The "@cuda.jit" decorator which marks the function for "just in time" compiling to machine language. The rng_states input is required for use with the xoroshiro128+ RNG. "thread_id" tells the GPU to assign each thread to a different position in the output array. Additional options exist for more complicated computations. 

In [39]:
@cuda.jit
def splitting_probability_gpu(rng_states, out):
    thread_id = cuda.grid(1)
    x=1/2
    while x<1 and x>0:
        x += 0.01 * xoroshiro128p_normal_float32(rng_states, thread_id)
        if x<0:
            out[thread_id] = 1

As a benchmark test, we run both implementations for the same number of trials, saving outputs and total run times. Note that the invocation of the CUDA kernel designates the total number of threads using "[blocks, threads_per_block]".

In [40]:
#t0 = time.time()
#out_cpu = np.zeros(trials, dtype=np.float32)
#for i in range(trials):
#    out_cpu[i] = splitting_probability_cpu()
#run_time_cpu = time.time() - t0

t0 = time.time()
rng_states = create_xoroshiro128p_states(threads_per_block * blocks, seed=np.random.randint(1, high = 1000))
out_gpu = np.zeros(threads_per_block * blocks, dtype=np.float32)
splitting_probability_gpu[blocks, threads_per_block](rng_states, out_gpu)
run_time_gpu = time.time() - t0

/home/jake/.local/lib/python3.10/site-packages/numba/cuda/cudadrv/devicearray.py:886: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


We now print the results of the comparison.

In [41]:
mean_cpu = np.mean(out_cpu)
mean_gpu = np.mean(out_gpu)

print("Number of trials: ", str(trials))
print("CPU splitting probability: ", str(mean_cpu))
print("GPU splitting probability: ", str(mean_gpu))
print("CPU total computation time: ", str(run_time_cpu))
print("GPU total computation time: ", str(run_time_gpu))
print("GPU speed increase: ", str(run_time_cpu / run_time_gpu))

Number of trials:  99840
CPU splitting probability:  0.5021587
GPU splitting probability:  0.5021234
CPU total computation time:  29.281405687332153
GPU total computation time:  0.23433232307434082
GPU speed increase:  124.95675075112351
